In [11]:
import pandas as pd

from kommun import Kommun

df = pd.read_csv("information-files/befolkningsdata/2014M1-2018M9.csv", sep=";")

kommuner: list[Kommun] = []
    


Unnamed: 0
Unnamed: 1
0114 Upplands Väsby
0115 Vallentuna
0117 Österåker
0120 Värmdö
0123 Järfälla
0125 Ekerö
0126 Huddinge
0127 Botkyrka
0128 Salem
0136 Haninge
0138 Tyresö
0139 Upplands-Bro
0140 Nykvarn
0160 Täby
0162 Danderyd
0163 Sollentuna
0180 Stockholm
0181 Södertälje
0182 Nacka
0183 Sundbyberg
0184 Solna
0186 Lidingö
0187 Vaxholm
0188 Norrtälje
0191 Sigtuna
0192 Nynäshamn
0305 Håbo
0319 Älvkarleby
0330 Knivsta
0331 Heby
0360 Tierp
0380 Uppsala
0381 Enköping
0382 Östhammar
0428 Vingåker
0461 Gnesta
0480 Nyköping
0481 Oxelösund
0482 Flen
0483 Katrineholm
0484 Eskilstuna
0486 Strängnäs
0488 Trosa
0509 Ödeshög
0512 Ydre
0513 Kinda
0560 Boxholm
0561 Åtvidaberg
0562 Finspång
0563 Valdemarsvik
0580 Linköping
0581 Norrköping
0582 Söderköping
0583 Motala
0584 Vadstena
0586 Mjölby
0604 Aneby
0617 Gnosjö
0642 Mullsjö
0643 Habo
0662 Gislaved
0665 Vaggeryd
0680 Jönköping
0682 Nässjö
0683 Värnamo
0684 Sävsjö
0685 Vetlanda
0686 Eksjö
0687 Tranås
0760 Uppvidinge
0761 Lessebo
0763 Tingsryd
0764